In [1]:
# importing necessary libraries
import scipy.io
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from numpy import asarray
from tqdm import tqdm
import re
from PIL import Image
import pywt

In [2]:
# specified data and label directory
data_dir = '/home/kiara/Desktop/EEG_Combined_Project/dataset/'
# specifiying file names for dataset
file_names = os.listdir(data_dir)
file_names = ["p4c2.mat", "p4c3.mat", "p4c4.mat", "p5c1.mat", "p5c2.mat", "p5c3.mat", "p5c4.mat", "p6c1.mat", "p6c2.mat", "p6c3.mat", "p6c4.mat", "p8c1.mat", "p8c2.mat", "p8c3.mat", "p8c4.mat", "p9c1.mat", "p9c2.mat", "p9c3.mat", "p9c4.mat", "p10c1.mat", "p10c2.mat", "p10c3.mat", "p10c4.mat", "p11c1.mat", "p11c2.mat", "p11c3.mat", "p11c4.mat", "p12c1.mat", "p12c2.mat", "p12c3.mat", "p12c4.mat", "p13c1.mat", "p13c2.mat", "p13c3.mat", "p13c4.mat", "p15c1.mat", "p15c2.mat", "p15c3.mat", "p15c4.mat", "p16c1.mat", "p16c2.mat", "p16c3.mat", "p16c4.mat", "p17c1.mat", "p17c2.mat", "p17c3.mat", "p17c4.mat", "p18c1.mat", "p18c2.mat", "p18c3.mat", "p18c4.mat", "p20c1.mat", "p20c2.mat", "p20c3.mat", "p20c4.mat", "p21c1.mat", "p21c2.mat", "p21c3.mat", "p21c4.mat", "p23c1.mat", "p23c2.mat", "p23c3.mat", "p23c4.mat", "p24c1.mat", "p24c2.mat", "p24c3.mat", "p24c4.mat"]

In [3]:
# choosing a file for data extraction
file_title = 16
file_name1 = file_names[file_title] 
# generating random number for random selection of train and test set
m = re.search('p(.+?)c', file_name1)
if m:
    random_state_value = int(m.group(1))

In [4]:
# obtaining the data as numpy array
load_file1 = os.path.join(data_dir, file_name1)
data1 = scipy.io.loadmat(load_file1)
my_data = data1['A']
# obtaining timestamps at which the labels switch
my_label1 = data1['latency']

In [5]:
# obtaining timestamps at which the labels switch as a list with 0 as the initial value
my_label1_shape = [0]
for i in range(my_label1.shape[1]):
    my_label1_shape.append(my_label1[0][i])

In [6]:
# choosing a limiting value to facilitate label creation
limiter = len(my_label1_shape)
# obtaining dimensions of the data
dims1 = my_data.shape[0]
dims2 = my_data.shape[1]

In [7]:
# preparing labels as a numpy array
Y_train = np.zeros((len(my_label1_shape)), dtype = np.float32)
label_zero = 0
label_one = 1
for i in range(0, limiter): # this runs from 0 to 17
    if (i<(limiter-1)):
        if (i==0 and (i != limiter-1)):
            Y_train[i] = label_zero
        
        elif (i%2!=0 and i!=0 and i != (limiter-1)):
            Y_train[i] = label_one
        
        elif ((i%2==0) and (i!=0) and (i != (limiter-1))):
            Y_train[i] = label_zero
            
    elif (i >= (limiter-1)):
        #print("this loop is executing")
        #print(i)
        if (i%2==0):
            Y_train[i] = label_zero
        elif (i%2!=0):
            Y_train[i] = label_one

In [8]:
# obtaining a label list with end and begining time stamps included
l1=my_label1_shape
l1.append(dims2)
# a list with durationof each time stamp for data
res = [l1[i + 1] - l1[i] for i in range(len(l1)-1)] 

In [9]:
X_train = np.zeros((Y_train.shape[0], dims1, max(res)), dtype = np.float32)
# creating X_train with 0 padded to make all the columns of the same dimension
for i in range(Y_train.shape[0]): # loop chalega 0 se 17 tak
    if (i<(Y_train.shape[0]-1)):
        arr1 = my_data[0:dims1, l1[i]:l1[i+1]]
        to_pad = max(res) - arr1.shape[1]
        X_add = np.pad(arr1, ((0,0),(0,to_pad)))
        X_train[i] = X_add
    elif (i==(Y_train.shape[0]-1)):
        arr1 = my_data[0:dims1, l1[i]:dims2]
        to_pad = max(res) - arr1.shape[1]
        X_add = np.pad(arr1, ((0,0),(0,to_pad)))
        X_train[i] = X_add
    else:
        print("----------------------------ERROR HAS OCCURED--------------------------------------")
        print(file_name1)

In [10]:
from csv import writer
def append_list_as_row(file_name, list_of_elem):
    # open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # creating writer object from csv module
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)
        
def wavelet_transform(input_data):
    coeff, freq = pywt.cwt(input_data, X_train.shape[0], "morl")
    return coeff  

In [11]:
my_X = wavelet_transform(X_train)
x_train = my_X[0].reshape(my_X.shape[1], my_X.shape[2], my_X.shape[3], -1)

# initializing log directory and model name
m = re.search('p(.+?).m', file_name1)
log_directory = 'Log_wavelet_'+str(file_name1)
model_name = 'model_wavelet_'+str(file_name1)+'.h5'

In [12]:
# performing train test split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(x_train, Y_train, test_size=0.1)

In [13]:
INPUT_SHAPE = (X_Train.shape[1], X_Train.shape[2], X_Train.shape[3]) 

In [14]:
# Defining the model
model = Sequential()
#l = [32, 64]
'''
l = [64, 128]
model.add(Conv2D(l[0], (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(l[0], (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(l[1], (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
'''


model.add(Flatten(input_shape=INPUT_SHAPE))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 963008)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               123265152 
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0

In [15]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 500, monitor = 'val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir = log_directory)
]

In [16]:
# model training
results = model.fit(X_Train, Y_Train, validation_split = 0.2, batch_size = 4, epochs = 200, verbose=0, callbacks=callbacks)

In [17]:
# model evaluation on test data
w_scores = model.evaluate(X_Test, Y_Test, verbose=1)

1/1 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - acc: 1.0000


In [18]:
# predicting test data values
Y_hat = model.predict(X_Test)
Y_pred = []
for item in Y_hat:
    if item>=(np.mean(Y_hat)):
        Y_pred.append(1)
    else:
        Y_pred.append(0)

In [19]:
# obtaining accuracy values
correct = 0
for i in range(len(Y_pred)):
    if Y_Test[i] == Y_pred[i]:
        correct += 1
wavelet_accuracy = correct/len(Y_pred)

In [20]:
print("Accuracy from Wavelet")
print(wavelet_accuracy)

Accuracy from Wavelet
1.0


In [21]:
#row_contents = [file_name1, w_scores[1], wavelet_accuracy]
#row_contents = ['Participant', 'Accuracy_Wavelet W Score', 'Calculated Accuracy']
#append_list_as_row('/home/kiara/Desktop/Wavelet2.csv', row_contents)